In [16]:
from numpy import loadtxt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt

In [19]:
# load the dataset
df = pd.read_excel (r'patient_data.xlsx', sheet_name='Sheet1')
raw_datas = df.values.tolist()
print(raw_datas[0],raw_datas[1])


features_list = list(df.to_dict().keys())[:-5] 
features_res_list = list(df.to_dict().keys())[-5:] 
print("features : \n", features_list)
print("features res: \n", features_res_list)

print(df[features_list[0]].tolist())

['M', 56, 'Fr', 'white', 'urban>1 milion', 'plane', 'professional school', 5, 'no', 'N', 'N', 'N', 'Y', 'N'] ['M', 47, 'Fr', 'white', '<100 000', 'hills', 'high school', 7, 'no', 'N', 'N', 'N', 'N', 'N']
features : 
 ['sex', 'age', 'country of birth', 'race', 'place of living (aggomeration) ', 'geographie of living place', 'profession (studies)', 'Berg scale (profession)', 'recent change of professional field']
features res: 
 ['It is recommended to have a screening test for cervical (uterin) cancer every 3 years with cervical cytology test ', 'It is recommended to have a screening test every 3 years wit cervical cytology test alone', 'it is recommended to have a screening test every 5 years with high-risk human papilloma virus (PV) test alone', 'It is recommended to have a screening stool test every 2 years: the high senzitivity guaiac fecal occult blood test (gFOBT) or fecal immunochemical test (FIT)', 'It is recommended to have a mammography every 2 years']
['M', 'M', 'F', 'M', 'F',

In [18]:
# split into input (X) and output (y) variables
X=[]
y=[]
y_element1 = []
y_element2 = []
y_element3 = []
y_element4 = []
y_element5 = []

poss_list = {}
cnt_list = [0]*len(raw_datas[0])

for j in range(len(raw_datas)):
    X_element = []
    
    for i in range(len(raw_datas[0])):
           
        if (i<=len(raw_datas[0])-6):
            ##on est dans les features
            X_element.append(raw_datas[j][i])
    
        elif (i==len(raw_datas[0])-1) :
            if (raw_datas[j][i]=='Y') :
            
                y_element5.append(1)
            else :
                ##par défaut si on a pas de réponse je met 0 comme pour 'N'
                y_element5.append(0)
        elif (i==len(raw_datas[0])-2) :
            if (raw_datas[j][i]=='Y') :
            
                y_element4.append(1)
            else :
                ##par défaut si on a pas de réponse je met 0 comme pour 'N'
                y_element4.append(0)
        elif (i==len(raw_datas[0])-3) :
            if (raw_datas[j][i]=='Y') :
            
                y_element3.append(1)
            else :
                ##par défaut si on a pas de réponse je met 0 comme pour 'N'
                y_element3.append(0)
        elif (i==len(raw_datas[0])-4) :
            if (raw_datas[j][i]=='Y') :
            
                y_element2.append(1)
            else :
                ##par défaut si on a pas de réponse je met 0 comme pour 'N'
                y_element2.append(0)
        elif (i==len(raw_datas[0])-5) :
            if (raw_datas[j][i]=='Y') :
            
                y_element1.append(1)
            else :
                ##par défaut si on a pas de réponse je met 0 comme pour 'N'
                y_element1.append(0)
        
    
    X.append(X_element)   

y.append(y_element1)  
y.append(y_element2)
y.append(y_element3)
y.append(y_element4)
y.append(y_element5)

print(X)
print(y)

[['M', 56, 'Fr', 'white', 'urban>1 milion', 'plane', 'professional school', 5, 'no'], ['M', 47, 'Fr', 'white', '<100 000', 'hills', 'high school', 7, 'no'], ['F', 52, 'Fr', 'white', 'urban<1 milion', 'sea', 'bachelor', 3, 'Y'], ['M', 53, 'Fr', 'white', '<100 000', 'plane', 'bachelor', 5, 'Y'], ['F', 34, 'Fr', 'white', '> 1 million ', 'plane', 'master', 3, 'Y'], ['F', 27, 'Fr', 'white', '<100 000', 'hills', 'professional school', 7, 'No'], ['F', 46, 'Fr', 'white', '> 1 million ', 'sea', 'master', 2, 'No'], ['F', 39, 'Fr', 'white', '> 1 million ', 'plane', 'bachelor', 3, 'No'], ['M', 62, 'Fr', 'white', '100 000-1 milion', 'sea', 'professional school', 5, 'Y'], ['M', 28, 'African', 'black', '100 000-1 milion', 'plane', 'bachelor', 3, 'Y'], ['F', 44, 'African', 'black', '> 1 million ', 'sea', 'high school', 6, 'No'], ['F', 48, 'African', 'black', '<100 000', 'sea', 'high school', 5, 'Y'], ['F', 39, 'African', 'black', '100 000-1 milion', 'plane', 'bachelor', 4, 'Y'], ['M', 40, 'Afro-americ

In [15]:
# define the keras model
dimension = len(X[0])

# define two sets of inputs
inputA = Input(shape=(32,))
inputB = Input(shape=(128,))

# the first branch operates on the first input
x = Dense(8, activation="relu")(inputA)
x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)

# the second branch opreates on the second input
y = Dense(64, activation="relu")(inputB)
y = Dense(32, activation="relu")(y)
y = Dense(4, activation="relu")(y)
y = Model(inputs=inputB, outputs=y)

# combine the output of the two branches
combined = concatenate([x.output, y.output])

# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)

# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input], outputs=z)

In [12]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)

ValueError: Data cardinality is ambiguous:
  x sizes: 29
  y sizes: 5
Make sure all arrays contain the same number of samples.

In [14]:
# make class predictions with the model
predictions = (model.predict(X) > 0.5).astype(int)

ValueError: in user code:

    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\fifid\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibilit